In [1]:
# import relevant packages

import pandas as pd
import os
import re

from html import unescape
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)

## Load raw datasets

In [2]:
df_dict = dict()

PATH = "../0_data/main/0_raw"

for file in sorted(os.listdir(PATH)):
    if "ipynb" not in file:
        print(re.sub('\.csv$', '', file))
        df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/{file}")

bas19_es
dyn21_en
for19_pt
has19_hi
has20_hi
has21_hi
ous19_ar
ous19_fr
san20_it


## Reformat columns
Need separate logic for different datasets. 1 is for hateful, 0 for non-hateful.

In [3]:
# Dynabench 2021 / English
df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)

# Fortuna 2019 / Portuguese
df_dict["for19_pt"].rename(columns={"hatespeech_comb": "label"}, inplace=True)

# Basile 2019 / Spanish
df_dict["bas19_es"].rename(columns={"HS": "label"}, inplace=True)

# Sanguinetti 2020 / Italian
df_dict["san20_it"].rename(columns={"hs": "label"}, inplace=True)

# Ousidhoum 2019 / Arabic & French
for d in ["ous19_ar", "ous19_fr"]:
    df_dict[d]["label"] = df_dict[d].sentiment.apply(lambda x: 1 if "hateful" in x else 0)
    # text was already cleaned in a way that conflicts with our later cleaning, so we align it here
    df_dict[d]["text"] = df_dict[d].tweet.apply(lambda x: x.replace("@url", "http"))
    
# HASOC 19, 20 and 21 / Hindi
for d in ["has19_hi", "has20_hi", "has21_hi"]:
    df_dict[d]["label"] = df_dict[d].task_2.apply(lambda x: 1 if x=="HATE" else 0)
    
# drop redundant columns
for dataset in df_dict:
    if "split" in df_dict[dataset].columns:
        df_dict[dataset] = df_dict[dataset][["text", "label", "split"]]
    else:
        df_dict[dataset] = df_dict[dataset][["text", "label"]]

## Clean text

In [4]:
def clean_text(text):
    text = unescape(text)
    text = re.sub(r"@[A-Za-z0-9_-]+",'@user',text) # format expected by XLM-T
    text = re.sub(r"http\S+",'http',text) # format expected by XLM-T
    text = re.sub(r"\n",' ',text)
    text = re.sub(r"\t",' ',text)
    text = text.replace("[URL]", "http") # format expected by XLM-T
    text = text.strip()
    return text

for dataset in df_dict:
    df_dict[dataset]["text"] = df_dict[dataset].text.apply(lambda x: clean_text(x))

## Show descriptive stats

In [5]:
def descriptive_stats(df):
    n_total = df.shape[0]
    n_hate = df.label.sum()
    print("{} entries, of which {} ({:.2%}) are hateful.".format(n_total, n_hate, n_hate/n_total))
    return df.label.sum()/len(df), len(df)

for dataset in df_dict:
    print(dataset.upper())
    descriptive_stats(df_dict[dataset])
    display(df_dict[dataset].sample(5))
    print()

BAS19_ES
6600 entries, of which 2739 (41.50%) are hateful.


,text,label
3792,"1cosa que no entiendo es porque un tío puede enseñar la polla en redes sociales pero si una tía enseña las tetas,el chichi o lo que sea se ve mal Es así o me equivoco? Porque creo que no,creo que todavía falta cierta libertad o mentalidad abierta mejor dicho",0
406,Acá nos huevean si nos encapuchamos pero en los paises árabes estariamos todas obligadas a encapucharnos 🤷‍♀️,0
5535,Me gusta Gigi Hadid y no busco atacarla pero si yo subo una foto de mi axila sin depilar me tacharian de guarra y p… http,0
5723,"@user La concha de tu madre gordo hijo de puta!! La hija de puta de ti mujer es la que no pacífica las cosas,gordo falopero!",1
2375,le tengo que enseñar a mi perra a cerrar la puerta porque la abre y no se da cuenta de quE TENGO EL AIRE PRENDIDO,0



DYN21_EN
41144 entries, of which 22175 (53.90%) are hateful.


,text,label,split
40917,black coffees are disgusting,0,test
18986,"I don't think they are as attracted to ol' Hook Nose than they are to his MONEY. He also talks a good game, knowing what to say and when to say it. Case in point: I had to listen to an Jewish coworker waffle on and on about his wealth and how ""cultured"" he was. Decked out in high end clothes, metro-sexual hair doo, and frequently reminding us lowly Goyim that he has an advanced degree. He was clearly trolling for female coworkers. Some were actually falling for it. Luckily most of us could see straight through the charade, and we were able to talk these ladies out of associating with this guy.",1,train
40762,ch!nks shouldn't be allowed to adopt,1,test
1028,Why dont you go back to Africa?,1,dev
19296,These black women are big as this black whale,1,train



FOR19_PT
5670 entries, of which 1788 (31.53%) are hateful.


...

,text,label
835,معروف خنزير من يومه,0
780,كلمن يبرر لقتل روح بشر، هو كائن وسخ ومريض، ومشارك بالقتل، وماكو اي استثناء. #تاره_فارس,0
1954,يعني ياشويت خنازير الشرف ملوش معني عندكم يعني انتم مفيش عندكم كرمه بسببكم ياشويت زبالة الناس اخدت فكرة عن ستات مصر… http,0
438,@user @user كيف تقولون على ترامب خنزير اونتم عندكم اكبر قاعدة عسكرية له؟! الله يلعن حليب الحمير الي سوا في عقولكم كذا,0
2079,يا كفار قريش يا شاربي بول البعير,1



OUS19_FR
4014 entries, of which 399 (9.94%) are hateful.


,text,label
3994,@user @user fake en plus c'est de très mauvaise qualité personne ne s'exprime comme un attardé… http,0
2766,la préférence islamiste de la gauche est motivée par le clientélisme mais également par l’anticléricalisme.,0
3055,putain d'attardé,0
1239,@user @user la rétorique préferée de l'extreme droite. je suis pas raciste j'ai un ami arabe. ca ne marche… http,0
2431,donc j’étais dans le métro en train de coller mon nez contre mon écran comme un putain d’attardé seul avec frust… http,0



SAN20_IT
8100 entries, of which 3388 (41.83%) are hateful.


,text,label,split
3880,@user si ma oggi i nemici della nostra civilta sono i musulmani http,1,train
3775,"A quanto pare, il corano, oltre che in bagno si può usare anche per questo. Ma pare non funzioni molto bene..... http",1,train
7738,"#Facciamorete: i figli non servono ad esser famiglia. #LGBT: il sesso che dà figli non val più di anale e dildo. #Gay: i figli si comprano. #Femministe: si possono uccidere, se embrioni. #Pd: son sostituibili cogli stranieri. #Burioni: sono infetti. Evviva la cultura dell'#amore.",0,test
4401,MALEDETTISSIME MMERDE..... POI PERÒ I PROFUGHI DEL CAZZO VANNO SOCCORSI E OSPITATI NEGLI ALBERGHI CON IL WIFI... VI AUGURO TUTTO IL MALE POSSIBILE,1,train
6653,@user @user @user @user @user Ma perché non andate a Milano ad aiutare i loro stranieri nella stessa situazione? Ah no lì ci sono i compagni,0,train


## Create and export splits

In [6]:
# set aside 2k from each dataset for testing and 500 for dev
# except for Ousidhoum in French and Arabic, where train set would otherwise be too small
# and for HASOC 20 and 21 in Hindi, where test splits are given

TEST_SIZE = 2000
DEV_SIZE = 500

for dataset in df_dict:
    if "ous19_fr" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 2000, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1500.csv", index=False)
    elif "ous19_ar" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 1300, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1000, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_300.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_1000.csv", index=False)
    elif "has19_hi" in dataset or "has20_hi" in dataset: # use provided test sets
        df_dict[dataset][df_dict[dataset]["split"]=="test"].to_csv(f"../0_data/main/1_clean/{dataset}/test_{len(df_dict[dataset][df_dict[dataset]['split']=='test'])}.csv", index=False)
        df_dict[dataset], devset = train_test_split(df_dict[dataset][df_dict[dataset]["split"]=="train"], test_size = 500, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_500.csv", index=False)
    else:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = TEST_SIZE+DEV_SIZE, random_state=123)
        devset, testset = train_test_split(devtest, test_size = TEST_SIZE, random_state=123)
        devset.to_csv(f"../0_data/main/1_clean/{dataset}/dev_{DEV_SIZE}.csv", index=False)
        testset.to_csv(f"../0_data/main/1_clean/{dataset}/test_{TEST_SIZE}.csv", index=False)
        
# export all data that is not test or dev, so we can use it for full sample training
for dataset in df_dict:
    df_dict[dataset].to_csv(f"../0_data/main/1_clean/{dataset}/train_{len(df_dict[dataset])}.csv", index=False)

In [7]:
# create differently-sized train portions from rest of data

SEEDS = 10 # for repeated experiments with different random data selection
N_RANGE = [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000]

for dataset in df_dict:
    print(dataset.upper())
    for n in N_RANGE:
        
        # save all splits for Dynabench
        if n<len(df_dict[dataset]) and "dyn21" in dataset: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
        
        # save splits up to 2k for other datasets
        elif n<len(df_dict[dataset]) and n<=2000: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):  
                df_dict[dataset].sample(n, random_state = random_state).to_csv(f"../0_data/main/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
    
    print()

BAS19_ES
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set

DYN21_EN
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set
  saving n = 3000 training set
  saving n = 4000 training set
  saving n = 5000 training set
  saving n = 10000 training set
  saving n = 20000 training set

FOR19_PT
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 